# 1、Data 模块

In [18]:
## Data
import random 
import numpy as np
import torch
import math

zidian_x = '<SOS>,<EOS>,<PAD>,0,1,2,3,4,5,6,7,8,9,q,w,e,r,t,y,u,i,o,p,a,s,d,f,g,h,j,k,l,z,x,c,v,b,n,m' #定义所有字符集
zidian_x = {word: i for i, word in enumerate(zidian_x.split(','))} #将字符集做成字典和索引
zidian_xr = [word for word,i in zidian_x.items()] #zidian_x包含的字符

zidian_y = {Word.upper():i for Word, i in zidian_x.items()}
zidian_yr = [word for word, i in zidian_y.items()]

print("zidian_x: {}\n zidian_y: {}".format(zidian_x,zidian_y))

#获得句子和翻译后的句子，思路是将输入的句子逆序，同时小写字母翻译成大写字母，数字翻译成10-i
def get_data():
    #定义词集合
    words = [
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'q', 'w', 'e', 'r',
        't', 'y', 'u', 'i', 'o', 'p', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k',
        'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm'
    ]
    
    #定义每个词出现的频率,其实也可以随机生成
#     p = np.array([
#         1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
#         13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26
#     ])
    p = np.random.rand(1,36)
    p = p / p.sum()
    
    #随机选n个词组成句子
    n = np.random.randint(30, 48) #生成长度为30-48个词的句子
    x = np.random.choice(words, size = n, replace=True,p=p[0])
    x = x.tolist()
    
    #字母小写转大写，数字10-i
    y = []
    for k in x:
        if k.isdigit():
            y.append(str(9-int(k)))
        else:
            y.append(k.upper())
                  
    y = y + [y[-1]] #认为给y增加一个字符，使y的维度是x+1，目的是在后面预测字符时能用到第一个字符（）这部分后续理解再说
    
    #逆序
    y = y[::-1]
    
    #加上首位符号
    x = ['<SOS>'] + x + ['<EOS>']
    y = ['<SOS>'] + y + ['<EOS>']
    
    #补固定长度pad
    x = x + ['<PAD>']*50
    y = y + ['<PAD>']*51
    x = x[:50]
    y = y[:51]
    
    #编码成索引数据
    x = [zidian_x[k] for k in x]
    y = [zidian_y[k] for k in y]
    
    #转tensor
    x = torch.LongTensor(x)
    y = torch.LongTensor(y)                     
    
    return x,y

class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        super(Dataset, self).__init__()

    def __len__(self):
        return 100000

    def __getitem__(self, i):
        return get_data()

# 数据加载器
loader = torch.utils.data.DataLoader(dataset=Dataset(),
                                     batch_size=8,
                                     drop_last=True,
                                     shuffle=True,
                                     collate_fn=None)   

zidian_x: {'<SOS>': 0, '<EOS>': 1, '<PAD>': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, 'q': 13, 'w': 14, 'e': 15, 'r': 16, 't': 17, 'y': 18, 'u': 19, 'i': 20, 'o': 21, 'p': 22, 'a': 23, 's': 24, 'd': 25, 'f': 26, 'g': 27, 'h': 28, 'j': 29, 'k': 30, 'l': 31, 'z': 32, 'x': 33, 'c': 34, 'v': 35, 'b': 36, 'n': 37, 'm': 38}
 zidian_y: {'<SOS>': 0, '<EOS>': 1, '<PAD>': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, 'Q': 13, 'W': 14, 'E': 15, 'R': 16, 'T': 17, 'Y': 18, 'U': 19, 'I': 20, 'O': 21, 'P': 22, 'A': 23, 'S': 24, 'D': 25, 'F': 26, 'G': 27, 'H': 28, 'J': 29, 'K': 30, 'L': 31, 'Z': 32, 'X': 33, 'C': 34, 'V': 35, 'B': 36, 'N': 37, 'M': 38}


### 生成一个句子编码

In [2]:
wx,wy = get_data()
print(wx,wy)

tensor([ 0, 10, 20,  8, 31,  6, 35, 34, 20, 23, 37, 16, 29, 18, 37, 13, 27, 31,
        31, 33, 27, 23,  7, 29, 25,  8, 33, 29, 38, 38,  8, 29, 20,  6,  8, 35,
         7, 35, 19,  9,  9, 24,  1,  2,  2,  2,  2,  2,  2,  2]) tensor([ 0, 24, 24,  6,  6, 19, 35,  8, 35,  7,  9, 20, 29,  7, 38, 38, 29, 33,
         7, 25, 29,  8, 23, 27, 33, 31, 31, 27, 13, 37, 18, 29, 16, 37, 23, 20,
        34, 35,  9, 31,  7, 20,  5,  1,  2,  2,  2,  2,  2,  2,  2])


# 2、功能 模块

### Mask 模块

In [3]:
#mask的单列pad设置
def mask_pad(data):
    #此时句子还未embed
    #data = [b, 50] b句话50词
    #判断每个词是不是<PAD>
    mask = data == zidian_x['<PAD>']
    
    # [b, 50] -> [b, 1, 1, 50] 
    # 有什么用？现在是每句话有一个1*50的向量表示，在后面对每句话里词与词之间进行词间注意力计算需要生成50*50的词矩阵，
    # 第2个一是在多头注意力模块中会分成4个头，Q*K得到的score的数据格式是[b,4,50,50]（可继续往下看），因此为了对齐多设置一维
    
    mask = mask.reshape(-1, 1, 1, 50)
    
    # 在计算注意力时,是计算50个词和50个词相互之间的注意力,所以是个50*50的矩阵
    # 是pad的列是true,意味着任何词对pad的注意力都是0
    # 但是pad本身对其他词的注意力并不是0，且防止归一化计算出错
    # 所以是pad的行不是true
    
    # 复制n次
    # [b, 1, 1, 50] -> [b, 1, 50, 50]
    mask = mask.expand(-1, 1, 50, 50)
    return mask

#上三角pad设置，这部分是decoder预测中防止信息泄露用
def mask_tril(data):
    # 50*50的矩阵表示每个词对其他词是否可见
    # 上三角矩阵,不包括对角线,意味着,对每个词而言,他只能看到他自己,和他之前的词,而看不到之后的词
    # [1, 50, 50]
    """
    [[0, 1, 1, 1, 1],
     [0, 0, 1, 1, 1],
     [0, 0, 0, 1, 1],
     [0, 0, 0, 0, 1],
     [0, 0, 0, 0, 0]]"""
    
    tril = 1- torch.tril(torch.ones(1,50,50, dtype=torch.long))
     # 判断y当中每个词是不是pad,如果是pad则不可见
    # [b, 50]
    mask = data == zidian_y['<PAD>']
   
    # 变形+转型,为了之后的计算
    # [b, 1, 50]
    mask = mask.unsqueeze(1).long()
    
    # mask和tril求并集
    # [b, 1, 50] + [1, 50, 50] -> [b, 50, 50]
    mask = mask + tril
    """
    [[0, 1, 1, 1, 1],
     [0, 0, 1, 1, 1],
     [0, 0, 1, 1, 1],
     [0, 0, 1, 0, 1],
     [0, 0, 1, 0, 0]] 
    """
    
    # 转布尔型
    mask = mask > 0

    # 转布尔型,增加一个维度,便于后续的计算,[b, 50, 50]->[b, 1, 50, 50]
    mask = (mask == 1).unsqueeze(dim=1)

    return mask
    

### multi-head attention 模块

In [4]:
def attention(Q, K, V, mask):
    #QKV = [b,4,50,8]
    #b 句话，每句话50词，每个词编码32维向量，4个头，每个头分为8维向量
    # Q * K = [b,4,50,8] * [b,4,8,50] -> [b, 4, 50,50]
    score = torch.matmul(Q, K.permute(0, 1, 3, 2)) #得到50*50的注意力得分矩阵，表示每个词与其他词之间的相似度或重要性
    
    # 除以每个头维数的平方根,做数值缩放，防止内积过大
    score /= 8 ** 0.5
    
    # mask遮盖,mask是true的地方都被替换成-inf,这样在计算softmax的时候,-inf会被压缩到0
    # mask = [b, 1, 50, 50]
    score = score.masked_fill_(mask, -float('inf'))
    score = torch.softmax(score, dim=-1) # softmax对score归一化后mask的位置变为0,
    
    #S*V 得到每个词相对其他词的注意力值
    #[b,4,50,50]*[b,4,50,8] -> [b,4,50,8]
    score = torch.matmul(score, V)
    
    #将4个头合并
    #[b,4,50,8] -> [b,50,32]
    score = score.permute(0,2,1,3).reshape(-1,50,32)
    
    return score

1、先计算Q*K得到50*50的注意力得分矩阵，表示每个词与其他词之间的相似度或重要性

2、mask矩阵也是50*50, 将其中true的位置在score矩阵中替换，保证<PAD>补句子的部分不参与注意力计算
    
3、计算S*V得到最后的attention表示

In [5]:
class MultiHead(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_Q = torch.nn.Linear(32,32) #每个词32维
        self.fc_K = torch.nn.Linear(32,32)
        self.fc_V = torch.nn.Linear(32,32)
        
        self.out_fc = torch.nn.Linear(32,32)
        
        self.norm = torch.nn.LayerNorm(normalized_shape=32, elementwise_affine = True)#标准化
        self.dropout = torch.nn.Dropout(p=0.1)
    
    def forward(self, Q, K, V, mask):
        #b句话，每句50词，每个词编成32维向量
        #有Q,K,V = [b,50,32]

        b = Q.shape[0]
        
        #保留原始Q，后续短接用，（为什么只有Q？）
        clone_Q = Q.clone()
        
        #规范化，这块论文中是在线性运算后进行，但实验证明放在之前效果会更好
        Q = self.norm(Q)
        K = self.norm(K)
        V = self.norm(V)
        
        #线性运算，维度不变
        K = self.fc_K(K)
        V = self.fc_V(V)
        Q = self.fc_Q(Q)
        
        #拆分多头，32维分4头拆分维8维
        #[b,50,32] -> [b,4,50,8]
        Q = Q.reshape(b, 50, 4, 8).permute(0, 2, 1, 3)
        K = K.reshape(b, 50, 4, 8).permute(0, 2, 1, 3)
        V = V.reshape(b, 50, 4, 8).permute(0, 2, 1, 3)
        
        #计算注意力
        #[b,4,50,8] -> [b,50,32]
        score = attention(Q, K, V, mask)
        
        # 计算输出,维度不变
        # [b, 50, 32] -> [b, 50, 32]
        score = self.dropout(self.out_fc(score)) #防止过拟合
        # 短接
        score = clone_Q + score
        return score

### 位置编码


In [6]:
class PositionEmbedding(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        #pos是第几个词，i是第几个维度，d_model是维度总数
        def get_pe(pos, i, d_model):
            fenmu = 1e4**(i/d_model)
            pe = pos / fenmu
            
            if i%2 == 0:
                return math.sin(pe)
            return math.cos(pe)
        
        #初始化位置编码矩阵
        pe = torch.empty(50,32)
        for i in range(50):
            for j in range(32):
                pe[i,j] = get_pe(i, j, 32)
        pe = pe.unsqueeze(0) #给矩阵添加一个中括号升维
        
        #定义为不更新的常量
        self.register_buffer('pe', pe)
        
        #词编码层
        self.embed = torch.nn.Embedding(39, 32) #zidian_x所有的词有39个词，编码位置有39个且为32维
        #初始化参数
        self.embed.weight.data.normal_(0, 0.1)
    
    def forward(self, x):
        #[b, 50] -> [b, 50, 32]
        embed = self.embed(x) #将b句话传进来进行词编码
        
        #词编码和位置编码相加
        #[b,50,32]+[1,50,32] -> [b,50,32]
        embed = embed + self.pe
        
        return embed

### 全连接层 feed_forward()

In [7]:
class FullyConnectedOutput(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=32, out_features=64),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=64, out_features=32),
            torch.nn.Dropout(p=0.1),
        )
        
        self.norm = torch.nn.LayerNorm(normalized_shape=32,
                                       elementwise_affine=True)
        
    def forward(self, x):
        clone_x = x.clone()

        x = self.norm(x)

        #全连接运算
        #[b,50,32] -> [b,50,32]
        out = self.fc(x)

        #短接
        out = clone_x + out
        return out 

# 3、model模块

### encoder

In [8]:
class EncoderLayer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mh = MultiHead()
        self.fc = FullyConnectedOutput()
        
    def forward(self, x, mask):
        # 计算自注意力,维度不变
        # [b, 50, 32] -> [b, 50, 32]
        score = self.mh(x, x, x, mask)
        
        # 全连接输出,维度不变
        # [b, 50, 32] -> [b, 50, 32]
        out = self.fc(score)
        
        return out

class Encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = EncoderLayer()
        self.layer_2 = EncoderLayer()
        self.layer_3 = EncoderLayer()
        
    def forward(self, x, mask):
        x = self.layer_1(x, mask)
        x = self.layer_2(x, mask)
        x = self.layer_3(x, mask)

        return x
        
        

### decoder

In [9]:
class DecoderLayer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.mh1 = MultiHead()
        self.mh2 = MultiHead()
        self.fc = FullyConnectedOutput()
        
    def forward(self, x, y, mask_pad_x, mask_tril_y ):
        # 先计算y的自注意力,维度不变
        # [b, 50, 32] -> [b, 50, 32]

        y = self.mh1(y, y, y, mask_tril_y)
        # 结合x和y的注意力计算,维度不变
        # [b, 50, 32],[b, 50, 32] -> [b, 50, 32]
        y = self.mh2(y, x, x, mask_pad_x)
        # 全连接输出,维度不变
        # [b, 50, 32] -> [b, 50, 32]
        out = self.fc(y)

        return out
        
class Decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = DecoderLayer()
        self.layer_2 = DecoderLayer()
        self.layer_3 = DecoderLayer()
        
    def forward(self, x, y, mask_pad_x, mask_tril_y):
        y = self.layer_1(x, y, mask_pad_x, mask_tril_y)
        y = self.layer_2(x, y, mask_pad_x, mask_tril_y)
        y = self.layer_3(x, y, mask_pad_x, mask_tril_y)
        return y        

### Transformer

In [10]:
class Transformer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed_x = PositionEmbedding()
        self.embed_y = PositionEmbedding()
        self.encoder = Encoder()
        self.decoder = Decoder()
        
        self.fc_out = torch.nn.Linear(32,39)
        
    def forward(self, x, y):
        #[b, 1, 50, 50]
        mask_pad_x = mask_pad(x)
        mask_tril_y = mask_tril(y)
        
        #embedding编码，添加位置信息
        # x = [b, 50] -> [b, 50, 32]
        # y = [b, 50] -> [b, 50, 32]
        x, y = self.embed_x(x), self.embed_y(y)
        
        #encoder
        x = self.encoder(x, mask_pad_x)
        
        #decoder
        y = self.decoder(x, y, mask_pad_x, mask_tril_y)
        
        #全连接输出
        #[b,50,32] -> [b,50,39]        
        y = self.fc_out(y)
        
        return y

# 5、main测试 

In [19]:
model = Transformer()
loss_func = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=2e-3)
sched = torch.optim.lr_scheduler.StepLR(optim, step_size=3, gamma=0.5)

for epoch in range(1):
    for i, (x, y) in enumerate(loader):
        #x = [8, 50]
        #y = [8, 51] batchsize 为8
        
        # 在训练时,是拿y的每一个字符输入,预测下一个字符,所以不需要最后一个字
        # [8, 50, 39]
        pred = model(x, y[:, :-1])

        #[8, 50, 39] -> [400, 39]
        pred = pred.reshape(-1, 39)
        
        #[8, 51] -> [400, 39]
        y = y[:, 1:].reshape(-1)

        #忽略pad
        select = y != zidian_y['<PAD>']
        pred = pred[select]
        y = y[select]
        
        loss = loss_func(pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if i % 200 == 0:
            
            pred = pred.argmax(1)
            
            correct = (pred == y).sum().item()
            accuracy = correct / len(pred)
            lr = optim.param_groups[0]['lr']
            print(epoch, i, lr, loss.item(), accuracy)
            if i %10000 == 0:
                print("y:{}\npred:{} \nsum:{}".format(y,pred,(pred == y).sum()))
    sched.step()
    
#test


0 0 0.002 3.968126058578491 0.02476780185758514
y:tensor([34, 34, 21, 23, 35, 26,  5, 26,  6, 16, 36, 35, 21, 23, 19, 32, 23, 23,
        11,  5, 23, 17,  5, 37, 12, 29, 28,  5, 21, 19, 21,  7,  1,  7,  7, 23,
         5,  8, 17, 15, 33, 29, 16, 22, 12, 35, 16, 24, 30, 11,  8, 28,  8, 11,
        37, 26, 10, 36,  7,  4, 35, 26, 11,  6, 14, 21,  6, 32, 37, 14, 38, 32,
         9, 37, 15, 24, 26, 27, 13,  1,  4,  4,  9,  9, 13,  9, 29, 10, 13, 12,
        23, 10, 28, 20, 34,  6, 35, 22, 18, 21, 20,  9, 35, 10, 30,  4, 36, 28,
        18, 37, 22, 18, 38, 38, 21, 18,  4,  1, 14, 14, 23, 33, 11, 30, 14, 16,
        14, 10, 22, 17, 24, 11, 10, 18, 22,  4, 35, 11, 24, 26, 21, 22, 17,  4,
         3, 21, 20, 26, 11, 26, 13, 14, 37, 28, 19, 20,  3, 17, 23,  1, 32, 32,
        36, 16, 21, 32, 36, 33, 24, 27,  6,  9,  8, 24, 16, 25, 33, 20, 37, 19,
         9, 10, 32, 23, 20,  3, 17,  6, 13,  4,  7, 13, 32, 20, 16, 12, 28,  8,
        18,  6, 18, 26, 28, 24,  1, 11, 11, 23, 15, 26,  3,  4, 32, 14

0 10200 0.002 0.037764277309179306 0.9878787878787879
0 10400 0.002 0.018591322004795074 0.9970845481049563
0 10600 0.002 0.19074751436710358 0.948948948948949
0 10800 0.002 0.09645926207304001 0.9797101449275363
0 11000 0.002 0.035375095903873444 0.990625
0 11200 0.002 0.021488884463906288 0.9942528735632183
0 11400 0.002 0.024565162137150764 0.9968454258675079
0 11600 0.002 0.036881834268569946 0.99079754601227
0 11800 0.002 0.030462488532066345 0.9907407407407407
0 12000 0.002 0.07674966007471085 0.9795918367346939
0 12200 0.002 0.068138986825943 0.9849397590361446
0 12400 0.002 0.006951395887881517 1.0


In [20]:
torch.save(model.state_dict(), './transformer.pth')

In [52]:
def predict(x):
    # x = [1, 50]
    model.eval()

    # [1, 1, 50, 50]
    mask_pad_x = mask_pad(x)

    # 初始化输出,这个是固定值
    # [1, 50]
    # [[0,2,2,2...]]
    target = [zidian_y['<SOS>']] + [zidian_y['<PAD>']] * 49
    target = torch.LongTensor(target).unsqueeze(0)

    # x编码,添加位置信息
    # [1, 50] -> [1, 50, 32]
    x = model.embed_x(x)

    # 编码层计算,维度不变
    # [1, 50, 32] -> [1, 50, 32]
    x = model.encoder(x, mask_pad_x)

    # 遍历生成第1个词到第49个词
    for i in range(49):
        # [1, 50]
        y = target

        # [1, 1, 50, 50]
        mask_tril_y = mask_tril(y)

        # y编码,添加位置信息
        # [1, 50] -> [1, 50, 32]
        y = model.embed_y(y)

        # 解码层计算,维度不变
        # [1, 50, 32],[1, 50, 32] -> [1, 50, 32]
        y = model.decoder(x, y, mask_pad_x, mask_tril_y)

        # 全连接输出,39分类
        # [1, 50, 32] -> [1, 50, 39]
        out = model.fc_out(y)

        # 取出当前词的输出
        # [1, 50, 39] -> [1, 39]
        out = out[:, i, :]

        # 取出分类结果
        # [1, 39] -> [1]
        out = out.argmax(dim=1).detach()

        # 以当前词预测下一个词,填到结果中
        target[:, i + 1] = out

    return target

In [82]:
# x = [zidian_x['<SOS>']]+[zidian_x['m'],zidian_x['g'],zidian_x['d']] +[zidian_x['<EOS>']]+ [zidian_x['<PAD>']]*50
x = ['<SOS>','m','f','j','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','c','<EOS>'] + ['<PAD>']*50
x = x[:50]
# print(x)
x = [zidian_x[k] for k in x]
x = [ 0, 37, 36,  7, 15, 16, 26,  8, 30,  8, 33, 25, 31,  3,  6, 34, 15, 15,
         25, 19,  8, 25, 19, 31, 37, 10, 23, 23, 33, 36,  4, 34, 34, 16,  6, 25,
          5,  3, 35, 19, 15, 22, 10,  1,  2,  2,  2,  2,  2,  2]

x = torch.LongTensor(x)
print(x)
print(''.join([zidian_xr[i] for i in x.tolist()]))
out = predict(x)
print(out)
print(''.join([zidian_yr[i] for i in out[i].unsqueeze(0)[0].tolist()]))


tensor([ 0, 37, 36,  7, 15, 16, 26,  8, 30,  8, 33, 25, 31,  3,  6, 34, 15, 15,
        25, 19,  8, 25, 19, 31, 37, 10, 23, 23, 33, 36,  4, 34, 34, 16,  6, 25,
         5,  3, 35, 19, 15, 22, 10,  1,  2,  2,  2,  2,  2,  2])
<SOS>nb4erf5k5xdl03ceedu5duln7aaxb1ccr3d20vuep7<EOS><PAD><PAD><PAD><PAD><PAD><PAD>
tensor([[ 0,  5,  5, 22, 15, 19, 35, 12, 10, 25,  9, 16, 34, 34, 11, 36, 33, 23,
         23,  5, 37, 31, 19, 25,  7, 19, 25, 15, 15, 34,  9, 12, 31, 25, 33,  7,
         30,  7, 26, 16, 15,  8, 36, 37,  1,  1,  1,  1,  1,  1]])
<SOS>22PEUV97D6RCC8BXAA2NLUD4UDEEC69LDX4K4FRE5BN<EOS><EOS><EOS><EOS><EOS><EOS>


该训练比较简单，若文本过短效果不好，只能找到长序列的特征

In [70]:
#正式测试
for i, (x, y) in enumerate(loader):
    print(type(i))
    break
    
for i in range(8):
    print(i)
    print(''.join([zidian_xr[i] for i in x[i].tolist()]))
    print(''.join([zidian_yr[i] for i in y[i].tolist()]))
    print(''.join([zidian_yr[i] for i in predict1(x[i].unsqueeze(0))[0].tolist()]))

<class 'int'>
